## Algoritmos de Búsqueda
### Plan de Reasignación de Carteras

En este cuaderno se resuelve el cambio de una cartera actual a una cartera objetivo de una forma
alternativa.  Se plantea como encontrar la secuencia de pasos u "operaciones" que hay que realizar.
En este caso incluiremos las opciones de comprar, vender o traspasar fondos de inversión

In [ ]:
import numpy as np 
import pandas as pd
import simpleai as ai
from collections import defaultdict

In [ ]:
from simpleai.search import SearchProblem, astar, breadth_first

### Datos 
Tenemos una lista de fondos para nuestro universo, con la característica que nos indica si son traspasables o no

In [ ]:
asset_df = pd.read_csv('../data/asset_data.csv', index_col=0)
asset_df

Definimos de ejemplo la cartera actual y la cartera objetivo 

In [ ]:
port_init = {
    'FR0010693051': 5000,
    'IE00B66F4759': 4000,
    'IE00B2NPKV68': 3000,
    'LU0318940003': 2000,
}
port_goal = {
    'FR0010693051': 2500,
    'LU0318940003': 4500,
    'IE00B4ND3602': 2500,
    'LU0582533245': 3500,
}

___

### Representación del Problema
    - Estados: la cartera en cada nodo
    - Acciones: la operación ejecutada para cambiar de estado
    - Problema: instancia de SearchProblem con nuestro problema 

In [ ]:
class FundAction:
    """
    Clase que define una operación
        tag: texto para identificar la operación
        money: guarda la cantidad de la operacion
        orig: el fondo de origen (vacio para compras)
        dest: el fondo de destino. (vacio para ventas)
    """
    def __init__(self, tag, money, orig='', dest=''):
        self.tag = tag
        self.orig = orig
        self.money = money
        self.dest = dest
        
    def __str__(self):
        return f'[{self.tag} {self.money} {self.orig} {self.dest}]'
    
    def __repr__(self):
        return f'[{self.tag} {self.money} {self.orig} {self.dest}]'
        

In [ ]:
class PortState:
    """
    Clase que representa un estado del problema, o sea una cartera
    """
    def __init__(self, port_dict):
        self.funds = defaultdict(float, port_dict)
    def __str__(self):
        items = [f'{k}:{val}' for k, val in self.funds.items()]
        return '\n'.join(items)
    def __repr__(self):
        items = [f'{k}:{val}' for k, val in self.funds.items()]
        return '\n'.join(items)

In [ ]:
PortState(port_goal)

In [ ]:
class Rebalancer(SearchProblem):
    """
    Clase que representa nuestro problema de búsqueda
    Iniciamos con la cartera actual, la cartera objetivo y los
    datos del universo de fondos
    """
    def __init__(self, current, target, asset_data): 
        self.asset_data = asset_data
        
        current['_CASH_'] = 0
        curr_total = sum(current.values())
        target_total = sum(target.values())
        target['_CASH_'] = curr_total - target_total
        
        self.current = PortState(current)
        self.target = PortState(target)
        
        self.generados = 0
        SearchProblem.__init__(self, initial_state=self.current)
    
    def _act_traspasos(self, state):
        """
        funcion auxiliar que calcula los traspasos aplicables en un estado
        """
        # identificamos los traspasables de salida
        tras_out = {f: val for f, val in state.funds.items()
                    if f != '_CASH_' and self.asset_data.traspasable[f]}

        # identificamos los traspasables de entrada
        tras_in = {f: val for f, val in self.target.funds.items()
                   if  f != '_CASH_' and self.asset_data.traspasable[f]}
        
        t_actions = list()
        for f_from, val_from in tras_out.items():
            for f_to, val_to in tras_in.items():
                
                # solo si son distintos y la posicion de destino es mayor
                curr_to = state.funds[f_to]
                if f_from != f_to and val_to > curr_to:
                    delta = min(val_to - curr_to, val_from)
                    if val_from == delta:
                        act_tag = "TRASPASO"
                    else:
                        act_tag = "TRASPASO_PARCIAL"
                    action = FundAction(tag=act_tag,
                                        orig=f_from,
                                        money=delta,
                                        dest=f_to)
                    t_actions.append(action)       

        return t_actions
    
    def _act_sell(self, state):
        """
        funcion auxiliar que calcula las ventas aplicables en un estado
        """
        s_actions = list()
        
        for f, val in state.funds.items():
            if f == '_CASH_':
                continue
            target_val = self.target.funds[f]
            
            if val > target_val:
                delta = -(target_val - val)
                if f in self.target.funds.keys():
                    act_tag = 'VENTA PARCIAL'
                else:
                    act_tag = 'VENTA'
                
                action = FundAction(tag=act_tag,
                                    orig=f,
                                    money=delta)

                s_actions.append(action)
            
        return s_actions
            
    def _act_buy(self, state):
        """
        funcion auxiliar que calcula las compras aplicables en un estado
        """
        
        b_actions = list()
        for f, val in self.target.funds.items():
            if f == '_CASH_':
                continue
                
            # posicion actual o 0
            curr_val = state.funds[f]
            
            # si la posicion final es mayor y podemos comprarla ahora
            if (val > curr_val and state.funds['_CASH_'] > val - curr_val): 
                delta = val - curr_val
                   
                if curr_val == 0:
                    act_tag = "COMPRA" 
                else:
                    act_tag = "COMPRA PARCIAL"
                    
                action = FundAction(tag=act_tag,
                                    dest=f,
                                    money=delta)
                b_actions.append(action)
        return b_actions
        
    def actions(self, state):
        """
        Calcula todas las acciones aplicables en un estado a partir
        de las funciones auxiliares de cada tipo
        """
        succ = list()
      
        succ.extend(self._act_traspasos(state))
        succ.extend(self._act_sell(state))
        succ.extend(self._act_buy(state))
        
        return succ
    
    def result(self, state, action):
        """calcula el estado resultante de aplicar una acción a un estado"""
        
        new_state = PortState(state.funds.copy())
        
        orig = action.orig if action.orig != '' else '_CASH_'
        dest = action.dest if action.dest != '' else '_CASH_'
        
        new_state.funds[orig] = new_state.funds[orig] - action.money
        new_state.funds[dest] = new_state.funds[dest] + action.money
        
        self.generados += 1
        
        return new_state

    def is_goal(self, state):
        """
        identifica si un estado es la cartera objetivo
        """
        check_funds = {f: val for f, val in state.funds.items() 
                       if f != '_CASH_'}
        for f, val in check_funds.items():
            if self.target.funds[f] != val: 
                return False
        return True        
        
    def heuristic(self, state):
        """
        Calcula el minimo de operaciones hipotéticas calculadas de forma trivial
        Esto es, a partir de la diferencia con la meta, calcula la mitad de los
        fondos que no son 0.  Como mínimo tendriamos que hacer un traspaso para
        arreglas 2 discrepancias
        """
        serie_current = pd.Series(state.funds)
        serie_current.drop('_CASH_', inplace=True)
        serie_goal = pd.Series(self.target.funds)
        serie_goal.drop('_CASH_', inplace=True)
        
        diff_port = serie_goal - serie_current 
        real_diff = diff_port[diff_port != 0]
        return real_diff.shape[0]/2

In [ ]:
rebalancer = Rebalancer(port_init, port_goal, asset_df)

In [ ]:
rebalancer.current

vemos las acciones aplicables en el estado inicial

In [ ]:
aplicables = rebalancer.actions(rebalancer.current)
aplicables

In [ ]:
rebalancer.result(rebalancer.current, aplicables[0])

In [ ]:
rebalancer.heuristic(rebalancer.current)

In [ ]:
rebalancer.cost(rebalancer.current, aplicables[0], _)

In [ ]:
rebalancer.generados

___

### Resolución con algoritmo primero en amplitud

Resolvemos el problema ejecutando el algoritmo de búsqueda

In [ ]:
plan_result = breadth_first(rebalancer, graph_search=True)

Vemos el estado final y el plan de operaciones

In [ ]:
print(plan_result.state)
for i, i_state_action in enumerate(plan_result.path()):
    print(f'[{i}] {i_state_action[0]}')

In [ ]:
rebalancer.generados

___

### Resolución con algoritmo A*

In [ ]:
rebalancer2 = Rebalancer(port_init, port_goal, asset_df)

In [ ]:
result2 = astar(rebalancer2, graph_search=True)

In [ ]:
print(result2.state)
for i, i_state_action in enumerate(result2.path()):
    print(f'[{i}] {i_state_action[0]}')

In [ ]:
rebalancer2.generados